### query orders files JSON format

In [0]:
df = spark.read.json("/Volumes/workspace/raw/operation_data/orders")
display(df)

In [0]:
%sql
SELECT * FROM json.`/Volumes/workspace/raw/operation_data/orders`;

### query orders files TEXT format

In [0]:
df = spark.read.text("/Volumes/workspace/raw/operation_data/orders")
display(df)

In [0]:
%sql
SELECT * FROM text.`/Volumes/workspace/raw/operation_data/orders`;

### query orders files to view table

In [0]:
df.writeTo('workspace.raw.v_orders').createOrReplace()

In [0]:
%sql
use catalog workspace;
use schema raw;

CREATE OR REPLACE VIEW workspace.raw.v_orders
AS
SELECT * FROM text.`/Volumes/workspace/raw/operation_data/orders`;

### transform JSON string to JSON object

In [0]:
from pyspark.sql import functions as F

orders_schema = '''STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>'''

df_orders = spark.table('workspace.raw.v_orders')

df_fixed_orders = (
    df_orders.select (
        F.regexp_replace("value", '"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "$1"').alias("fixed_value")
    )
)

df_with_schema = (
    df_fixed_orders.select (
        F.schema_of_json (F.col("fixed_value")).alias("schema")
    )
)

df_json_orders = (
    df_fixed_orders.select (
        F.from_json("fixed_value", orders_schema).alias("json_value")
    )
)

In [0]:
%sql
with orders_fixed as (
  select
    value,
    regexp_replace(value, '"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "\$1') as fixed_value
  from workspace.raw.v_orders
)
select from_json(fixed_value, 'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') as json_value
from orders_fixed

In [0]:
%sql
CREATE OR REPLACE VIEW workspace.raw.j_orders
AS
with orders_fixed as (
  select
    value,
    regexp_replace(value, '"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "\$1') as fixed_value
  from workspace.raw.v_orders
)
select from_json(fixed_value, 'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') as json_value
from orders_fixed

### access JSON object

In [0]:
%sql
select 
  json_value.order_id,
  json_value.order_status,
  json_value.payment_method,
  json_value.total_amount,
  json_value.transaction_timestamp,
  json_value.customer_id,
  explode(array_distinct(json_value.items)) as item
from workspace.raw.j_orders

### json to silver table schema

In [0]:
%sql
CREATE OR REPLACE VIEW workspace.silver.v_orders AS 
select 
  json_value.order_id,
  json_value.order_status,
  json_value.payment_method,
  json_value.total_amount,
  json_value.transaction_timestamp,
  json_value.customer_id,
  explode(array_distinct(json_value.items)) as item
from workspace.raw.j_orders

### order summary

In [0]:
df_order_summary = (
    df_orders
    .withColumn("order_month", F.date_format('transaction_timestamp', 'yyyy-MM'))
    .groupBy('order_month', 'customer_id')
    .agg(
        F.countDistinct('order_id').alias('total_orders'),
        F.sum('quantity').alias('total_items_bought'),
        F.sum(F.col('price') * F.col('quantity')).alias('total_amount')
    )
)

In [0]:
%sql
select
  date_format(transaction_timestamp, 'yyyy-MM') as order_month,
  customer_id,
  count(distinct order_id) as total_orders,
  sum(item.quantity) as total_items_bought,
  sum(item.price * item.quantity) as total_amount
from workspace.silver.v_orders
group by order_month, customer_id